# Evidencia 2

Nombres: Diego Lira García
        Andrea Medina Rico

In [ ]:
%pip install mesa==2.3.1 --quiet
%pip install matplotlib numpy pandas --quiet
%pip install seaborn --quiet

In [2]:
from mesa import Agent, Model
from mesa.space import SingleGrid
from mesa.time import SimultaneousActivation

# Haremos uso de ''DataCollector'' para obtener información de cada paso de la simulación.
from mesa.datacollection import DataCollector

# Librería para modificar parámetros --> Corre varias simulaciones
from mesa.batchrunner import batch_run

# matplotlib lo usaremos crear una animación de cada uno de los pasos del modelo.
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.animation as animation
plt.rcParams["animation.html"] = "jshtml"
matplotlib.rcParams['animation.embed_limit'] = 2**128

# Importamos los siguientes paquetes para el mejor manejo de valores numéricos.
import numpy as np
import pandas as pd

# Definimos otros paquetes que vamos a usar para medir el tiempo de ejecución de nuestro algoritmo.
import time
import datetime

## 1. Solución aleatoria

In [ ]:
class FirefighterAgent(Agent):